In [5]:
import pandas as pd
import numpy as np
import os

os.chdir('/Users/gianni/projects/mnk-cleaning-analysis/')

from clean import *

os.chdir('/Users/gianni/Google Drive/Bas Zahy Gianni - Games/Data')

In [6]:
oc = [
    'index', 'subject', 'color', 'gi', 'mi', 
    'status', 'bp', 'wp', 'response', 'rt',
    'time', 'mouse_t', 'mouse_x'
]

fc = [
    'subject', 'is_comp', 'color', 'status',
    'bp', 'wp', 'response', 'rt', 'gi', 'mi',
    'computer', 'human', 'time', 'a', 'b',
    'aval', 'bval', 'val'
]

mc = ['subject', 'color', 'bp', 'wp', 'response', 'rt']

class Data():
    """ Data is the primary object for holding experimental data. It also contains functions
        for the loading, cleaning, augmentation, and export of the data tables. """

    def __init__(self, folder):
        self.data = self.load(folder)

    def load_file(self, folder, file_name, mouse=False):
        """ Initial preparation of data for individual files """

        # load file, drop nuissance columns, remove non-observations
        drop_cols = ['index'] if mouse else ['index', 'mouse_t', 'mouse_x']
        data = pd.read_csv(folder + file_name, names=self.original_columns).drop(drop_cols, axis=1)
        drop_status = (data.status != 'dummy') &  (data.status != 'ready') & (data.status != 'draw offer')
        data = data.loc[drop_status, :].copy().reset_index(drop=True)

        # assign unique subject label (from filename) and create separate cols for humans and computers
        sub_filter = data.rt > 0
        comp_filter = data.rt == 0
        first_move_filter = (data.mi == 0) & (data.gi%2 == 0)
        second_move_filter = (data.mi == 1) & (data.gi%2 == 0)

        data.loc[data.rt > 0, 'subject'] = file_name[:-4]
        data.loc[:, 'human'] = file_name[:-4]
        data.loc[:, 'computer'] = np.nan
        data.loc[comp_filter, 'computer'] = data.loc[comp_filter, 'subject']
        data.loc[first_move_filter, 'computer'] = data.loc[second_move_filter, 'computer']
        data.loc[:, 'computer'] = data.loc[:, 'computer'].fillna(method='ffill')
        data.loc[0, 'computer'] = data.loc[1, 'computer']

        return data

    def load(self, folder):
        """ Calls other functions to corrale data and some support information """
        self.exp_name = folder
        files = os.listdir(folder + '/Raw/')    
        files = [f for f in files if f[-3:] == 'csv']
        self.subjects = [f[:-4] for f in files]
        self.subject_dict = odict(zip(self.subjects, np.arange(len(self.subjects))))
        data = pd.concat([self.load_file(folder + '/Raw/', f) for f in files]).reset_index(drop=True)
        data = self.clean(data)

        return data

    def clean(self, df):
        """ Performs further cleaning that can be done on all data collectively """

        # anonymize subjects
        sub_filter = df.rt > 0 # filter computers out
        df.loc[sub_filter, 'subject'] = df.loc[sub_filter, 'subject'].map(self.subject_dict)
        df.loc[:, 'human'] = df.loc[:, 'human'].map(self.subject_dict)

        # give computers identifiable names
        comp_filter = df.rt == 0
        df.loc[comp_filter, 'subject'] = df.loc[comp_filter, 'subject'].astype(int) + 1000
        df.loc[pd.notnull(df.computer), 'computer'] = df.loc[pd.notnull(df.computer), 'computer'].astype(int) + 1000

        # force remove response from board
        for i in df.loc[df.status != 'EVAL', :].index.values:
            if df.loc[i,"color"] == 0:
                l = list(df.loc[i,"bp"])
                l[df.loc[i, "response"]] = '0'
                df.loc[i,"bp"] = ''.join(l)
            else:
                l = list(df.loc[i,"wp"])
                l[df.loc[i,"response"]] = '0'
                df.loc[i,"wp"] = ''.join(l)

        # force correct colors
        count_pieces = lambda x: np.array([np.array(list(df.loc[i, x])).astype(int).sum() for i in df.index.values])
        df.loc[:, 'color'] = count_pieces('bp') - count_pieces('wp')
        df.loc[:, 'color'] = df.loc[:, 'color'].astype(int).astype(str)

        # add is_comp
        is_computer = lambda x: "0" if x > 0 else "1"
        df.loc[:, 'is_comp'] = df.loc[:, 'rt'].map(is_computer)

        # correct move index in games
        df.loc[df.status.isin(['playing', 'win', 'draw']), 'mi'] = df.loc[df.status.isin(['playing', 'win', 'draw']), 'mi'] - 1
        return df

    def export_individuals(self, folder):
        for s, i in self.subject_dict.items():
            c = self.data.human == i
            d = self.data.loc[c, :].reset_index(drop=True)
            d = d.reindex_axis(self.full_output_columns, axis=1)
            d.to_csv(folder + '/Clean/' + s + '.csv', index=False)

        return None

    def export_tasks(self, folder):
        # separate tables by task
        game_selector = self.data.loc[:, 'status'].isin(['playing', 'win', 'draw'])
        afc2_selector = self.data.loc[:, 'status'] == 'AFC2'
        eval_selector = self.data.loc[:, 'status'] == 'EVAL'

        game = self.data.loc[game_selector, :]
        afc2 = self.data.loc[afc2_selector, :]
        eva  = self.data.loc[eval_selector, :]
        print(eva.head(5))

        game.loc[:, self.full_output_columns].to_csv(folder + '/Clean/_summaries/all_games_all_fields.csv', index=False)
        game.loc[:, self.game_model_columns].to_csv(folder + '/Clean/_summaries/all_games_model_input.csv', header=False, index=False)
        game.loc[game.rt > 0, self.game_model_columns].to_csv(folder + '/Clean/_summaries/all_games_model_input_no_comp.csv', header=False, index=False)
        afc2.loc[:, self.full_output_columns].to_csv(folder + '/Clean/_summaries/all_afc2s_all_fields.csv', index=False)
        afc2.loc[:, self.afc2_model_columns].to_csv(folder + '/Clean/_summaries/all_afc2s_model_input.csv', header=False, index=False)
        eva.loc[:, self.full_output_columns].to_csv(folder + '/Clean/_summaries/all_evals_all_fields.csv', index=False)
        eva.loc[:, self.eval_model_columns].to_csv(folder + '/Clean/_summaries/all_evals_model_input.csv', header=False, index=False)

        return None

OSError: File b'util/objective_errors.csv' does not exist